In [1]:
# Cell 1: imports and global parameters
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns



In [4]:
import tensorflow as tf
import os

# ---- Configurations for paths ----
INSECT_DATA_DIR = r"/Users/imeshaaluvihare/Desktop/Research/Identify Diseases for Lavender plant/Model Training LeafDisease & InsectDiseas/Insect_dataset"  
MODEL_SAVE_PATH = r"/Users/imeshaaluvihare/Desktop/Research/Identify Diseases for Lavender plant/insect_model.keras"

# Image / training params
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
SEED = 42

# Checking if dataset folder exists 
if not os.path.exists(INSECT_DATA_DIR):
    print("Kavi their is an ERROR: Dataset folder NOT found at:")
    print(INSECT_DATA_DIR)
else:
    print("Kavi Good News Dataset folder found!")

    # Check if it has class subfolders
    classes = [d for d in os.listdir(INSECT_DATA_DIR) if os.path.isdir(os.path.join(INSECT_DATA_DIR, d))]

    if len(classes) == 0:
        print("Kavi Their Is an ERROR: No class folders found inside dataset directory!")
    else:
        print("Kavi Suba Aranchiyak Found class folders:", classes)

        # Try loading the dataset
        try:
            train_ds = tf.keras.utils.image_dataset_from_directory(
                INSECT_DATA_DIR,
                seed=SEED,
                image_size=IMG_SIZE,
                batch_size=BATCH_SIZE
            )

            print("SUCCESS: Dataset loaded correctly!")
            print("Number of classes detected:", len(train_ds.class_names))
            print("Class names:", train_ds.class_names)

        except Exception as e:
            print(" Failed to load dataset!")
            print(e)


Kavi Good News Dataset folder found!
Kavi Suba Aranchiyak Found class folders: ['7683246', '1035434', '4308789', '5873357', '8378220', '5023047', '4480200', '4308787', '5757120', '7639821', '5755075', '4475157', '6097878', '1036216', '6097882', '5755044', '4988516', '1035231', '4478679', '6097885', '6097871', '8417764', '1035864', '4472929', '4308786', '1036286', '5755027', '5755011', '1035290', '4475908', '5753725', '4473297', '1035208', '6097883', '4475707', '8563753', '5755080', '6097879', '5716408', '4308801', '5755562', '1036192', '4473320', '4479870', '4308806', '5716406', '9479706', '5755302', '4473974', '8104778', '1035578', '7642452', '4471071', '4472897', '5755969', '9206709', '1035929', '5873215', '4474169', '8068514', '9314786', '5757205', '5873271', '9414758', '1035185', '4308807', '4308800', '8019310', '4470765', '5716409', '4473319', '7917481', '9533851', '5873214', '5755950', '1036796', '4474998', '9444427', '5872109', '7641814', '4472849', '9346940', '4471269', '575598

2025-11-30 20:23:18.004722: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-11-30 20:23:18.004899: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-11-30 20:23:18.005056: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-11-30 20:23:18.005557: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-30 20:23:18.006194: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
# ---- Create training & validation datasets ----
try:
    train_dataset = tf.keras.utils.image_dataset_from_directory(
        INSECT_DATA_DIR,
        validation_split=0.2,
        subset="training",
        seed=SEED,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        label_mode="categorical"   
    )

    val_dataset = tf.keras.utils.image_dataset_from_directory(
        INSECT_DATA_DIR,
        validation_split=0.2,
        subset="validation",
        seed=SEED,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        label_mode="categorical"
    )

    print("Kavi Suba Aranchiyak! 🟢 Training & Validation datasets created successfully!")
    print(f"Training batches: {len(train_dataset)}")
    print(f"Validation batches: {len(val_dataset)}")

except Exception as e:
    print("Kavi Bad News! 🔴 ERROR loading the datasets:", e)


Found 29732 files belonging to 293 classes.
Using 23786 files for training.
Found 29732 files belonging to 293 classes.
Using 5946 files for validation.
Kavi Suba Aranchiyak! 🟢 Training & Validation datasets created successfully!
Training batches: 744
Validation batches: 186


In [8]:
# ---- Prefetch & Optimize the dataset ----
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)

print("Kavi Suba aranchiyak ....Dataset pipeline optimized with AUTOTUNE.")


Kavi Suba aranchiyak ....Dataset pipeline optimized with AUTOTUNE.


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications, optimizers
import os

# ---- Configurations ----
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
INSECT_DATA_DIR = r"/Users/imeshaaluvihare/Desktop/Research/Identify Diseases for Lavender plant/Model Training LeafDisease & InsectDiseas/Insect_dataset"

# Count number of classes based on subfolders
num_classes = len([d for d in os.listdir(INSECT_DATA_DIR) if os.path.isdir(os.path.join(INSECT_DATA_DIR, d))])
print("✅ Number of classes detected:", num_classes)

# -----------------------------
# Data augmentation
# -----------------------------
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.2),
    layers.RandomTranslation(0.05, 0.05)
], name="data_augmentation")

# -----------------------------
# Base model: EfficientNetB0 pretrained on ImageNet
# -----------------------------
base_model = applications.EfficientNetB0(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # freeze base for initial training

# -----------------------------
# Build full model
# -----------------------------
inputs = layers.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = applications.efficientnet.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)  # flatten features
x = layers.Dropout(0.3)(x)              # regularization
outputs = layers.Dense(num_classes, activation="softmax")(x)  # multi-class output

model = models.Model(inputs, outputs)

# -----------------------------
# Compile model
# -----------------------------
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


✅ Number of classes detected: 293


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 293)            │       375,333 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,424,904 (16.88 MB)

 Trainable params: 375,333 (1.43 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [13]:
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
SEED = 42

INSECT_DATA_DIR = r"/Users/imeshaaluvihare/Desktop/Research/Identify Diseases for Lavender plant/Model Training LeafDisease & InsectDiseas/Insect_dataset"

# Create train & validation datasets (80% / 20%)
train_ds = tf.keras.utils.image_dataset_from_directory(
    INSECT_DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    INSECT_DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Save class names before applying prefetch
class_names = train_ds.class_names

# Optimize pipeline
train_ds = train_ds.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

print("✅ Training and validation datasets are ready!")
print("Classes detected:", class_names)


Found 29732 files belonging to 293 classes.
Using 23786 files for training.
Found 29732 files belonging to 293 classes.
Using 5946 files for validation.
✅ Training and validation datasets are ready!
Classes detected: ['1035167', '1035185', '1035194', '1035195', '1035204', '1035208', '1035231', '1035290', '1035366', '1035434', '1035542', '1035551', '1035578', '1035864', '1035929', '1035931', '1036066', '1036128', '1036154', '1036192', '1036203', '1036216', '1036255', '1036286', '1036789', '1036796', '1036893', '1036899', '1036917', '1036958', '1037293', '1037319', '1037633', '4308786', '4308787', '4308789', '4308790', '4308800', '4308801', '4308804', '4308805', '4308806', '4308807', '4308811', '4308812', '4308815', '4470539', '4470555', '4470765', '4470801', '4471071', '4471113', '4471202', '4471235', '4471238', '4471269', '4472828', '4472849', '4472858', '4472884', '4472897', '4472900', '4472907', '4472913', '4472929', '4473127', '4473277', '4473297', '4473319', '4473320', '4473617', '

In [15]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# -----------------------------
# Callbacks for better training
# -----------------------------
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),  # stop early if no improvement
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1),  # reduce LR if stuck
    ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_accuracy', save_best_only=True, verbose=1)  # save best model
]

# -----------------------------
# Train the model (initial training with frozen base)
# -----------------------------
INITIAL_EPOCHS = 10

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=INITIAL_EPOCHS,
    callbacks=callbacks
)

# -----------------------------
# Fine-tuning: unfreeze some layers of base model
# -----------------------------
base_model.trainable = True

# Optionally, freeze first N layers to avoid destroying pre-trained features
fine_tune_at = 100  # freeze first 100 layers
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

FINE_TUNE_EPOCHS = 20
TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS

model.compile(
    optimizer=optimizers.Adam(1e-5),  # lower LR for fine-tuning
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=TOTAL_EPOCHS,
    initial_epoch=history.epoch[-1],
    callbacks=callbacks
)


Epoch 1/10


2025-11-30 20:40:27.719823: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-11-30 20:40:38.716285: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:58: Filling up shuffle buffer (this may take a while): 427 of 1000
2025-11-30 20:40:48.718256: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:58: Filling up shuffle buffer (this may take a while): 690 of 1000
2025-11-30 20:40:50.964425: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


744/744 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.0807 - loss: 4.4140
Epoch 1: val_accuracy improved from None to 0.24807, saving model to /Users/imeshaaluvihare/Desktop/Research/Identify Diseases for Lavender plant/insect_model.keras
744/744 ━━━━━━━━━━━━━━━━━━━━ 179s 201ms/step - accuracy: 0.1234 - loss: 3.8995 - val_accuracy: 0.2481 - val_loss: 3.1469 - learning_rate: 0.0010
Epoch 2/10


2025-11-30 20:43:36.441323: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:58: Filling up shuffle buffer (this may take a while): 325 of 1000
2025-11-30 20:43:47.173325: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:58: Filling up shuffle buffer (this may take a while): 574 of 1000
2025-11-30 20:43:54.782058: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


744/744 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.2294 - loss: 3.0888
Epoch 2: val_accuracy improved from 0.24807 to 0.30760, saving model to /Users/imeshaaluvihare/Desktop/Research/Identify Diseases for Lavender plant/insect_model.keras
744/744 ━━━━━━━━━━━━━━━━━━━━ 204s 225ms/step - accuracy: 0.2393 - loss: 3.0583 - val_accuracy: 0.3076 - val_loss: 2.8434 - learning_rate: 0.0010
Epoch 3/10


...

744/744 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step - accuracy: 0.2395 - loss: 2.9210
Epoch 12: ReduceLROnPlateau reducing learning rate to 1.9999999494757505e-06.

Epoch 12: val_accuracy did not improve from 0.41271
744/744 ━━━━━━━━━━━━━━━━━━━━ 422s 523ms/step - accuracy: 0.2530 - loss: 2.8535 - val_accuracy: 0.3066 - val_loss: 2.5821 - learning_rate: 1.0000e-05
Epoch 13/30


2025-11-30 21:28:25.709675: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:58: Filling up shuffle buffer (this may take a while): 353 of 1000
2025-11-30 21:28:38.980457: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


744/744 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step - accuracy: 0.2944 - loss: 2.6053
Epoch 13: val_accuracy did not improve from 0.41271
744/744 ━━━━━━━━━━━━━━━━━━━━ 378s 466ms/step - accuracy: 0.2941 - loss: 2.6101 - val_accuracy: 0.3317 - val_loss: 2.4540 - learning_rate: 2.0000e-06
Epoch 14/30


2025-11-30 21:34:43.668899: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:58: Filling up shuffle buffer (this may take a while): 466 of 1000
2025-11-30 21:34:50.843240: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


744/744 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 0.2123 - loss: 3.1768
Epoch 14: val_accuracy did not improve from 0.41271
744/744 ━━━━━━━━━━━━━━━━━━━━ 327s 405ms/step - accuracy: 0.2184 - loss: 3.0832 - val_accuracy: 0.0230 - val_loss: 7.1109 - learning_rate: 2.0000e-06


In [16]:
import tensorflow as tf

# Path where the trained model is saved
MODEL_SAVE_PATH = r"/Users/imeshaaluvihare/Desktop/Research/Identify Diseases for Lavender plant/insect_model.keras"

# Load the model
model = tf.keras.models.load_model(MODEL_SAVE_PATH)
print("✅ Insect model loaded successfully!")
model.summary()


✅ Insect model loaded successfully!


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 293)            │       375,333 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,383,985 (12.91 MB)

 Trainable params: 375,333 (1.43 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 750,668 (2.86 MB)

In [18]:
import os
import tensorflow as tf

# Path to the insect dataset
EVAL_DATA_DIR = r"/Users/imeshaaluvihare/Desktop/Research/Identify Diseases for Lavender plant/Model Training LeafDisease & InsectDiseas/Insect_dataset"

# Image parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Load evaluation dataset (no shuffling)
eval_ds = tf.keras.utils.image_dataset_from_directory(
    EVAL_DATA_DIR,
    labels="inferred",
    label_mode="categorical",   # For multi-class classification
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Print class names BEFORE prefetch
print("✅ Evaluation dataset ready!")
print("Class names:", eval_ds.class_names)

# Prefetch for performance
eval_ds = eval_ds.prefetch(buffer_size=AUTOTUNE)


Found 29732 files belonging to 293 classes.
✅ Evaluation dataset ready!
Class names: ['1035167', '1035185', '1035194', '1035195', '1035204', '1035208', '1035231', '1035290', '1035366', '1035434', '1035542', '1035551', '1035578', '1035864', '1035929', '1035931', '1036066', '1036128', '1036154', '1036192', '1036203', '1036216', '1036255', '1036286', '1036789', '1036796', '1036893', '1036899', '1036917', '1036958', '1037293', '1037319', '1037633', '4308786', '4308787', '4308789', '4308790', '4308800', '4308801', '4308804', '4308805', '4308806', '4308807', '4308811', '4308812', '4308815', '4470539', '4470555', '4470765', '4470801', '4471071', '4471113', '4471202', '4471235', '4471238', '4471269', '4472828', '4472849', '4472858', '4472884', '4472897', '4472900', '4472907', '4472913', '4472929', '4473127', '4473277', '4473297', '4473319', '4473320', '4473617', '4473795', '4473834', '4473860', '4473874', '4473903', '4473974', '4474146', '4474169', '4474653', '4474780', '4474861', '4474974', '

In [21]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    INSECT_DATA_DIR,
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",  # integer labels, shape (batch,)
    validation_split=0.2,
    subset="training"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    INSECT_DATA_DIR,
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",  # integer labels
    validation_split=0.2,
    subset="validation"
)


Found 29732 files belonging to 293 classes.
Using 23786 files for training.
Found 29732 files belonging to 293 classes.
Using 5946 files for validation.


In [23]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])


In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import os

# ---- Paths ----
INSECT_DATA_DIR = r"/Users/imeshaaluvihare/Desktop/Research/Identify Diseases for Lavender plant/Model Training LeafDisease & InsectDiseas/Insect_dataset"
MODEL_SAVE_PATH = r"/Users/imeshaaluvihare/Desktop/Research/Identify Diseases for Lavender plant/insect_model_improved.keras"

# ---- Training params ----
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
INITIAL_EPOCHS = 10       # start with 10 epochs
FINE_TUNE_EPOCHS = 20     # fine-tuning top layers
TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS


In [26]:
# Load dataset first
train_ds = tf.keras.utils.image_dataset_from_directory(
    INSECT_DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    INSECT_DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int"
)

# Save class names **before prefetching**
class_names = train_ds.class_names
print("Number of classes:", len(class_names))
print("Classes:", class_names[:10], "...")  # show first 10 for brevity

# Optimize datasets
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


Found 29732 files belonging to 293 classes.
Using 23786 files for training.
Found 29732 files belonging to 293 classes.
Using 5946 files for validation.
Number of classes: 293
Classes: ['1035167', '1035185', '1035194', '1035195', '1035204', '1035208', '1035231', '1035290', '1035366', '1035434'] ...


In [27]:
from tensorflow.keras import layers, models, applications, optimizers

# Number of classes
NUM_CLASSES = len(class_names)

# ---- 1. Load the base model (EfficientNetB0) without the top layer ----
base_model = applications.EfficientNetB0(
    include_top=False,       # exclude final Dense layer
    weights='imagenet',      # use pretrained weights
    input_shape=IMG_SIZE + (3,)
)
base_model.trainable = False   # freeze base model initially

# ---- 2. Build the classification head ----
inputs = layers.Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)  # keep batchnorm layers in inference mode
x = layers.GlobalAveragePooling2D()(x)  # flatten features
x = layers.Dropout(0.3)(x)              # regularization
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)  # multi-class softmax

# ---- 3. Combine base + head into a model ----
model = models.Model(inputs, outputs)

# ---- 4. Compile the model ----
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  # integer labels
    metrics=['accuracy']
)

# ---- 5. Summary ----
model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 293)            │       375,333 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,424,904 (16.88 MB)

 Trainable params: 375,333 (1.43 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [28]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# ---- Callbacks ----
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ModelCheckpoint(MODEL_SAVE_PATH, save_best_only=True, monitor='val_accuracy')
]

# ---- Train the model ----
INITIAL_EPOCHS = 10

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=INITIAL_EPOCHS,
    callbacks=callbacks
)


Epoch 1/10
744/744 ━━━━━━━━━━━━━━━━━━━━ 195s 249ms/step - accuracy: 0.3208 - loss: 3.0615 - val_accuracy: 0.5284 - val_loss: 2.0928
Epoch 2/10
744/744 ━━━━━━━━━━━━━━━━━━━━ 160s 215ms/step - accuracy: 0.5386 - loss: 1.9135 - val_accuracy: 0.6251 - val_loss: 1.5805
Epoch 3/10
744/744 ━━━━━━━━━━━━━━━━━━━━ 168s 225ms/step - accuracy: 0.6146 - loss: 1.5466 - val_accuracy: 0.6739 - val_loss: 1.3419
Epoch 4/10
744/744 ━━━━━━━━━━━━━━━━━━━━ 183s 246ms/step - accuracy: 0.6553 - loss: 1.3443 - val_accuracy: 0.6968 - val_loss: 1.1930
Epoch 5/10
744/744 ━━━━━━━━━━━━━━━━━━━━ 162s 217ms/step - accuracy: 0.6820 - loss: 1.2198 - val_accuracy: 0.7215 - val_loss: 1.1025
Epoch 6/10
744/744 ━━━━━━━━━━━━━━━━━━━━ 165s 221ms/step - accuracy: 0.7022 - loss: 1.1252 - val_accuracy: 0.7309 - val_loss: 1.0313
Epoch 7/10
744/744 ━━━━━━━━━━━━━━━━━━━━ 162s 217ms/step - accuracy: 0.7186 - loss: 1.0586 - val_accuracy: 0.7434 - val_loss: 0.9690
Epoch 8/10
744/744 ━━━━━━━━━━━━━━━━━━━━ 160s 215ms/step - accuracy: 0.7309 -

In [30]:
# ---- Evaluate the fine-tuned model ----
loss, accuracy = model.evaluate(val_ds)
print(f"\n📊 Fine-tuned Model Accuracy: {accuracy*100:.2f}%")
print(f"📉 Fine-tuned Model Loss    : {loss:.4f}")

# ---- Predictions for classification report & confusion matrix ----
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Get true labels
y_true = np.concatenate([y.numpy() for x, y in val_ds], axis=0)

# Predict
pred_probs = model.predict(val_ds)
pred_classes = np.argmax(pred_probs, axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, pred_classes))




186/186 ━━━━━━━━━━━━━━━━━━━━ 34s 174ms/step - accuracy: 0.7607 - loss: 0.8743

📊 Fine-tuned Model Accuracy: 76.07%
📉 Fine-tuned Model Loss    : 0.8743


2025-11-30 23:08:17.379790: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


186/186 ━━━━━━━━━━━━━━━━━━━━ 34s 180ms/step

Classification Report:
              precision    recall  f1-score   support

         141       0.00      0.00      0.00        15
         142       0.00      0.00      0.00        20
         143       0.00      0.00      0.00        31
         145       0.00      0.00      0.00         7
         147       0.00      0.00      0.00        34
         149       0.03      0.03      0.03        39
         151       0.00      0.00      0.00        20
         152       0.00      0.00      0.00        22
         153       0.00      0.00      0.00        42
         154       0.00      0.00      0.00        12
         155       0.00      0.00      0.00        35
         156       0.02      0.02      0.02        55
         157       0.00      0.00      0.00        41
         158       0.03      0.03      0.03        31
         159       0.00      0.00      0.00        12
         160       0.00      0.00      0.00        30
         161 

/Users/imeshaaluvihare/.pyenv/versions/3.10.15/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/imeshaaluvihare/.pyenv/versions/3.10.15/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/imeshaaluvihare/.pyenv/versions/3.10.15/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _war

In [31]:
# Save the trained model
model.save(MODEL_SAVE_PATH)
print(f"✅ Model saved successfully at: {MODEL_SAVE_PATH}")


✅ Model saved successfully at: /Users/imeshaaluvihare/Desktop/Research/Identify Diseases for Lavender plant/insect_model_improved.keras
